In [ ]:
#Import everything
from http import cookies
import selenium
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import time
import json
import urllib
import requests
import os
'''
def click_element(xpath: str):
    try:
        button = self.driver.find_element(By.XPATH, xpath)
        button.click()
    except TimeoutException:
        print("TimeoutException: Button not found")
'''

In [ ]:
# Open the website
driver = webdriver.Chrome()
driver.get('https://vndb.org/v')

In [ ]:
# Try out next page
nav_bar = driver.find_element(By.XPATH, '//*[@class="maintabs browsetabs "]')
button = nav_bar.find_element(By.XPATH, './ul[2]/li[1]/a')
button.click()
# Yeah!!!

In [ ]:
# Try out search field
search_field = driver.find_element(By.XPATH, '//*[@id="q"]')
search_field.clear()
search_field.send_keys('yellow')
search_button = driver.find_element(By.XPATH, '//*[@value="Search!"]')
search_button.click()

In [ ]:
# Get the first result in the search field
result_1 = driver.find_elements(By.XPATH, '/html/body/div[4]/form/div[3]/table/tbody/tr[3]/td[2]/img')
#print(result_1[3].get_attribute('title'))
table = driver.find_element(By.XPATH, '/html/body/div[4]/form/div[3]/table/tbody')
rows = table.find_elements(By.XPATH, './tr')
#print(rows[0].text)
row_1_elements = rows[0].find_elements(By.XPATH, './td')
print(row_1_elements[0].text)
print(row_1_elements[5].text.split()[1][1:-1])
# I find the columns to be:
# 1: Title of game
# 2: Platforms available (in images, need to be dealt with separately)
# 3: Languages available (in images, need to be dealt with separately)
# 4: Release date
# 5: Popularity index
# 6: Rating out of 10 (No. of users rated on VNDB)
platforms = row_1_elements[1].find_elements(By.XPATH, './img')
languages = row_1_elements[2].find_elements(By.XPATH, './abbr')
for platform in platforms:
    print(platform.get_attribute('title'))
for language in languages:
    print(language.get_attribute('title'))
# Get url to description page
description_page_url = row_1_elements[0].find_element(By.XPATH, './a').get_attribute('href')
print(description_page_url)

In [ ]:
# Now get all useful information from the table and store 'em in a dictionary
table = driver.find_element(By.XPATH, '/html/body/div[4]/form/div[3]/table/tbody')
rows = table.find_elements(By.XPATH, './tr')
table_data = [] # going to be a list of dictionaries
for row in rows[:5]:
    row_elements = row.find_elements(By.XPATH, './td')
    title = row_elements[0].text
    release_date = row_elements[3].text
    popularity = row_elements[4].text
    rating = row_elements[5].text.split()[0]
    no_of_voters = int(row_elements[5].text.split()[1][1:-1]) # need to take away brackets

    # Platforms and languages are displayed in images, need to be dealt with separately
    platforms = []
    languages = []
    platforms_img = row_elements[1].find_elements(By.XPATH, './img')
    languages_img = row_elements[2].find_elements(By.XPATH, './abbr')
    for platform in platforms_img:
        platforms.append(platform.get_attribute('title'))
    platforms = ", ".join(platforms)
    for language in languages_img:
        languages.append(language.get_attribute('title'))
    languages = ", ".join(languages)
    # Get description page link
    description_page_url = row_elements[0].find_element(By.XPATH, './a').get_attribute('href')

    row_data_dict = {"URL": description_page_url,
                     "Title": title,
                     "Platform_available": platforms,
                     "Language_available": languages,
                     "Release_date": release_date,
                     "Popularity": popularity,
                     "Rating": rating,
                     "No_of_voters": no_of_voters}
    table_data.append(row_data_dict)
print(table_data[3])

 
    

In [ ]:
# Now we get the data, we store them in pd dataframe and export
import pandas as pd
dataframe = pd.DataFrame(table_data)
dataframe.to_json('raw_data/data.json')
print(dataframe)

In [ ]:
# Download an image
import ssl
import datetime
ssl._create_default_https_context = ssl._create_unverified_context
driver.get(table_data[3]['URL'])
img_holder = driver.find_element(By.XPATH, '//*[@class="imghover--visible"]')
image = img_holder.find_element(By.XPATH, './img')
url = image.get_attribute('src')
print(url)
datetime_now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
image_id = url[23:].replace('/','') # trims https://s2.vndb.org/cv/
filepath = f'raw_data/images/{datetime_now}_{image_id}'
print(filepath)
print(image_id)
urllib.request.urlretrieve(url, filepath)
print(url[23:].replace('/',''))



In [ ]:
search_field = driver.find_element(By.XPATH, '//*[@id="q"]')
search_field.clear()
search_field.send_keys('black')
search_button = driver.find_element(By.XPATH, '//*[@value="Search!"]')
search_button.click()

In [3]:
import selenium

ModuleNotFoundError: No module named 'selenium'